## For : Full_ICD10_ATC.csv

#### Split Diagnose and Medication, and save new dataframe

In [ ]:
import re
import warnings
warnings.filterwarnings('ignore')
import pandas as pd
import numpy as np
import random
from IPython.display import display
from IPython.display import Image
import matplotlib.pyplot as plt
from sklearn.linear_model import LogisticRegression,LinearRegression
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score, mean_squared_error, accuracy_score,confusion_matrix,precision_score,recall_score,f1_score
from sklearn.feature_extraction.text import CountVectorizer
import matplotlib.pyplot as plt
import os
from dotenv import load_dotenv
load_dotenv()


class Full_ICD10_ATC_ClassifyReadmissionWithMedicationDiagnosis:
    def __init__(self, file1):
        self.file1 = file1
        self.merged_df = None
        self.selected_column_merged_df = None
        self.without_diag_medic_selected_column_merged_df = None
    
    def load_data(self):
        Full_ICD10_ATC = pd.read_csv(self.file1)
        self.merged_df = Full_ICD10_ATC        
        self.merged_df = self.merged_df[['episode_id', 'MiddleChildhood', 'Preschooler', 'Teenager', 'F', 'M',
       'num_diagnoses', 'num_medications', 'pasient', 'age', 'gender',
       'episode_order', 'islast', 'closingcode', 'aftercode',
       'episode_start_date', 'episode_end_date', 'tillnextepisode',
       'Length_of_Episode', 'Cat_LOE', 'Cat_TNE', 'Cat_LOE_desc',
       'Cat_TNE_desc', 'Count_visit', 'Cat_CV', 'Therapy_ratio',
       'Examination_ratio', 'Advisory_ratio', 'TreatmentPlanning_ratio',
       'Outpatient_ratio', 'Inpatient_ratio', 'Inpatient_day_ratio',
       'Inpatient_daynight_ratio', 'closingcode_0', 'closingcode_1',
       'closingcode_2', 'closingcode_3', 'closingcode_4', 'closingcode_5',
       'closingcode_6', 'closingcode_9', 'aftercode_1', 'aftercode_2',
       'aftercode_3', 'aftercode_4', 'aftercode_5', 'LOE_Norm', 'LOE_long',
       'LOE_short', 'No_more_episode', 'TNE_less180', 'TNE_more180',
       'Care_intensity', 'LOE_desc', 'TNE_desc', 'diagnoses',
       'actual_med_Full_ATC']]
        self.merged_df = self.merged_df.copy(deep=True)
        return self.merged_df

        
    def split_diagnosis_medication(self):
        newmerged_df = Full_ICD10_ATC_ClassifyReadmissionWithMedicationDiagnosis_Obj.load_data()
        columns = ['diagnoses', 'actual_med_Full_ATC']
        for col in columns:
            newmerged_df[col] = newmerged_df[col].apply(lambda d:[] if pd.isnull(d) else d)
            newmerged_df[col] = newmerged_df[col].str.replace("[", "")
            newmerged_df[col] = newmerged_df[col].str.replace("]", "")
            newmerged_df[col] = newmerged_df[col].str.replace("'", "")
            newmerged_df[col] = newmerged_df[col].str.replace(" ", "")
            newmerged_df[col] = newmerged_df[col].str.split(',')
            newmerged_df = newmerged_df.explode(col)
            
            # Show only top 100 columns names
            temp_store_100 = newmerged_df[col].value_counts()
            names_only = temp_store_100.index.tolist()
            print(names_only)
            
            original_columns = newmerged_df.columns.tolist()
            dummies = newmerged_df[col].str.get_dummies(sep=',')
            newmerged_df = pd.concat([newmerged_df, dummies], axis=1)
            newmerged_df = newmerged_df.drop(col, axis=1)
            
        newmerged_df.to_csv('Unique_Full_ICD10_ATC.csv',index=False)
        


Full_ICD10_ATC_ClassifyReadmissionWithMedicationDiagnosis_Obj = Full_ICD10_ATC_ClassifyReadmissionWithMedicationDiagnosis('/mnt/work/workbench/dipendrp/new-data/Full_ICD10_ATC.csv')
Full_ICD10_ATC_ClassifyReadmissionWithMedicationDiagnosis_Obj.split_diagnosis_medication()


### Select top 100 medication and diagnosis

In [ ]:
all_Unique_Full_ICD10_ATC = pd.read_csv('Unique_Full_ICD10_ATC.csv')
top100 = all_Unique_Full_ICD10_ATC[['episode_id', 'MiddleChildhood', 'Preschooler', 'Teenager', 'F', 'M', 'num_diagnoses', 'num_medications', 'pasient', 'age', 'gender', 'episode_order', 'islast', 'closingcode', 'aftercode', 'episode_start_date', 'episode_end_date', 'tillnextepisode', 'Length_of_Episode', 'Cat_LOE', 'Cat_TNE', 'Cat_LOE_desc', 'Cat_TNE_desc', 
 'Count_visit', 'Cat_CV', 'Therapy_ratio', 'Examination_ratio', 'Advisory_ratio', 'TreatmentPlanning_ratio', 'Outpatient_ratio', 'Inpatient_ratio', 'Inpatient_day_ratio', 'Inpatient_daynight_ratio', 'closingcode_0', 'closingcode_1', 'closingcode_2', 'closingcode_3', 'closingcode_4', 'closingcode_5', 'closingcode_6', 'closingcode_9', 
 'aftercode_1', 'aftercode_2', 'aftercode_3', 'aftercode_4', 'aftercode_5', 'LOE_Norm', 'LOE_long', 'LOE_short', 'No_more_episode', 'TNE_less180', 'TNE_more180', 'Care_intensity', 'LOE_desc', 'TNE_desc','3131', '29622', '3151', '313', '3001', '304', '312', '3132', '3133', '3009', '3052', '2781', '1004', '3003', '2972', '3152', '3153', '1019', '563', '2501', '315', '495', '7981', '637', '75021', '798', '476', 
'3034', '939', '345', '969', '339', '3004', '340', '7142', '5994', '2961', '3008', '300', '3891', '2926', '3069', '20411', '2953', '781', '785', '2903', '343', '915', '561', '3012', '316', '292', '3031', '74711', '2951', '317', '949', '3689', '3453', '301', '3678', '19111', '1002', '3781', '789', '3401', '327', '3679', '2911', '661', '3276', '3011', '3671', '296', '3033',
'389', '74712', '7522', '2713', '7581', '3274', '34512', '3123', '7373', '1702', '34511', '871', '74713', '3711', '819', '75211', '3591', '2444', '5551', '2649', '6562', '788', '2914', '1023', '5552', '302', '658', '2952', '3451', '733', '2801', '770', '3811', '1008', '62611', '262', '3311', '7512', '3895', '75112', '1015', '656', '760', '368', '930', '745', '7551', '759', 
'1994', '5641', '2532', '277', '7561', '7321', '1551', '7282', '499', '2612', '2614', '53011', '201', '1009', '2022', '5091', '306', '3672', '323', '3892', '2791', '69542', '5134', '3869', '2445', '758', '18911', '20421', '979', '3381', '396', '7554', '817', '2643', '2442', '736', '565', '3592', '55521', '347', '7061', '75121', '0704', '2924', '1028', '57181', '1701', 
'24521', '3695', '773', '1031', '1000', '5965', '6964', '0792', '5611', '835', '32741', '947', '496', '4332', '357', '42711', '079', '136', '3501', '3484', '7531', '2553', '535', '8033', '3684', '5302', '28731', '748', '6564', '506', '25311', '3275', '1911', '3894', '259', '6041', '3382', '756', '3621', '472', '7556', '7491', '3681', '532', '75561', '6563', '295', '1011', 
'830', '324', '27614', '818', '53014', '564', '2564', '802', '75011', '749', '981', '28811', '344', '4742', '4589','6264', '6491', '7492', '691', '1029', '0703', '260', '25011', '3782', '739', '3462', '5772', '8003', '6941', '558', '303', '2531', '341', '1034', '4338', '4181', '288', '7472', '2441', '4741', '1499', '7471', '365', '2701', '805', '591', '4801', '8032', '907',
'5305', '1033', '657', '1013', '2502', '1010', '5921', '634', '946', '2768', '2602', '471', '010', '510', '75013', '2922', '2921', '71011', '7169','N06BA04', 'N06AB06', 'N06BA09', 'A06BA04', 'N06AB03', 'N05CH01', 'R06AD01', 'N05AX08', 'N05AH04', 'N06BA12', 'N05AX12', 'N03AX09', 'NO6BAO4', 'N05AH03', 'NO6BA04', 'N06AB10', 'N05AA02', 'N06AX03', 'N05CF01', 'A11EA', 'A12AX', 'N06AX16', 
'N03AG01', 'N05BA01', 'N05AF03', 'N02CX02', 'N05CF02', 'R06AE07', 'N05BB01', 'N05BA04', 'N06AA04', 'N02CC01', 'G03AA07', 'N03AE01', 'G03AC09', 'N06AX11', 'N02CC03', 'A06AD65', 'G03AA09', 'N06AB04', 'NO6BAO9', 'H01BA02', 'N06AX12', 'B03AA07', 'N05CD02', 'A10BA02', 'A02BC05', 'C02AC02', 'A03FA01', 'N06AB05', 'N06BA02', 'S01AA01', 'D07BC01', 'D07AC13', 
'J01FA10', 'G03AA13', 'N01BB20', 'G03AC06', 'A06AD11', 'G03AA12', 'N05AD01', 'S01GX02', 'D10AD03', 'M01AE02', 'A02BA02', 'J01CF01', 'D07AA02', 'N02AA59', 'A02BC02', 'R01AD09', 'G03FB05', 'N03AX14', 'D07AB08', 'G03AD02', 'A02BC01', 'D07AC01', 'N02BA01', 'D01AC20', 'R01AC02', 'N03AF01',
'A12BA02', 'M01AE01', 'S01GX09', 'R05DA01', 'D06AX01', 'H02AB02', 'A12BA01', 'D07XC01', 'N02BE01', 'A11JB', 'S01AA13', 'C09AA02', 'C07AA05', 'J05AB01', 'D06AX05', 'G03CA03', 'G03AD01', 'N07BA03', 'D10AD53', 'D10AD01', 'A07AA02', 'N01BB02', 'N06AA09', 'D07AB02', 'J01CA08', 'N07BA02', 'D06BB03', 'R03BA05', 'D06AA03', 'R03AC03', 'G03HB01', 
'D09AA02', 'J01CE02', 'R03BA02', 'B03BA03', 'R06AX27', 'J07AH03']]

top100.to_csv('All_Full_ICD10_ATC.csv',index=False)

In [ ]:
top100 = pd.read_csv('All_Full_ICD10_ATC.csv')
print(top100.dtypes)


### Classification and Predicition

In [ ]:
import warnings
warnings.filterwarnings('ignore')

import pandas as pd
import numpy as np
import random
from IPython.display import display
from IPython.display import Image
import matplotlib.pyplot as plt

from sklearn.linear_model import LogisticRegression,LinearRegression
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score, mean_squared_error, accuracy_score,confusion_matrix,precision_score,recall_score,f1_score
from sklearn.metrics import classification_report

from sklearn.feature_extraction.text import CountVectorizer
import matplotlib.pyplot as plt

import os
from dotenv import load_dotenv
load_dotenv()


################## Without Medication and Diagnosis ##################
#                                                                    #
#                                                                    #
#                                                                    #
######################################################################

class ClassifyReadmissionWithMedicationDiagnosis:
    def __init__(self, file1):
        self.file1 = file1
        self.merged_df = None
        self.selected_column_merged_df = None
        self.without_diag_medic_selected_column_merged_df = None
    
    def load_data(self):
        original_df_1 = pd.read_csv(self.file1)
        self.merged_df = original_df_1[['MiddleChildhood', 'Preschooler', 'Teenager', 'F', 'M',
        'num_diagnoses','num_medications', 'Cat_LOE', 'Cat_CV', 'Therapy_ratio','Examination_ratio', 
        'Advisory_ratio', 'TreatmentPlanning_ratio','Outpatient_ratio', 'Inpatient_ratio', 
        'Inpatient_day_ratio','Inpatient_daynight_ratio', 'closingcode_0', 'closingcode_1',
        'closingcode_2', 'closingcode_3', 'closingcode_4', 'closingcode_5','closingcode_6', 
        'closingcode_9', 'aftercode_1', 'aftercode_2','aftercode_3', 'aftercode_4', 'aftercode_5',
        'LOE_Norm', 'LOE_long','LOE_short','Length_of_Episode', 'Count_visit','tillnextepisode','Cat_TNE','No_more_episode','TNE_more180','TNE_less180','Care_intensity','TNE_desc','3131', '29622', '3151', '313', '3001', '304', '312', '3132', '3133', '3009', '3052', '2781', '1004', '3003', '2972', '3152', '3153', 
       '1019', '563', '2501', '315', '495', '7981', '637', '75021', '798', '476', '3034', '939', '345', '969', '339', '3004', '340', '7142', '5994', '2961', '3008', '300', '3891', '2926', '3069', '20411', '2953', '781', '785', '2903', '343', '915', '561', '3012', '316', '292', '3031', '74711', '2951', '317', '949', '3689', 
       '3453', '301', '3678', '19111', '1002', '3781', '789', '3401', '327', '3679', '2911', '661', '3276', '3011', '3671', '296', '3033', '389', '74712', '7522', '2713', '7581', '3274', '34512', '3123', '7373', '1702', '34511', '871', '74713', '3711', '819', '75211', '3591', '2444', '5551', '2649', '6562', '788', '2914', '1023',
        'N06BA04', 'N06AB06', 'N06BA09', 'A06BA04', 'N06AB03', 'N05CH01', 'R06AD01', 'N05AX08', 'N05AH04', 'N06BA12', 'N05AX12', 'N03AX09', 'NO6BAO4', 'N05AH03', 'NO6BA04', 'N06AB10', 'N05AA02', 'N06AX03', 'N05CF01', 'A11EA', 'A12AX', 'N06AX16', 'N03AG01', 'N05BA01', 'N05AF03', 'N02CX02', 'N05CF02', 'R06AE07', 'N05BB01', 'N05BA04', 'N06AA04', 
        'N02CC01', 'G03AA07', 'N03AE01', 'G03AC09', 'N06AX11', 'N02CC03', 'A06AD65', 'G03AA09', 'N06AB04', 'NO6BAO9', 'H01BA02', 'N06AX12', 'B03AA07', 'N05CD02', 'A10BA02', 'A02BC05', 'C02AC02', 'A03FA01', 'N06AB05', 'N06BA02', 'S01AA01', 'D07BC01', 'D07AC13', 'J01FA10', 'G03AA13', 'N01BB20', 'G03AC06', 'A06AD11', 'G03AA12', 'N05AD01', 'S01GX02',
        'D10AD03', 'M01AE02', 'A02BA02', 'J01CF01', 'D07AA02', 'N02AA59', 'A02BC02', 'R01AD09', 'G03FB05', 'N03AX14', 'D07AB08', 'G03AD02', 'A02BC01', 'D07AC01', 'N02BA01', 'D01AC20', 'R01AC02', 'N03AF01', 'A12BA02', 'M01AE01', 'S01GX09', 'R05DA01', 'D06AX01', 'H02AB02', 'A12BA01', 'D07XC01', 'N02BE01', 'A11JB', 'S01AA13', 'C09AA02', 'C07AA05', 
        'J05AB01', 'D06AX05', 'G03CA03', 'G03AD01', 'N07BA03', 'D10AD53','D10AD01']]
       
        self.merged_df.loc[self.merged_df['tillnextepisode']==0,'tillnextepisode'] = 50000
        
        self.dependent_variable_tillnextepisode = self.merged_df[['tillnextepisode']]
        
        self.dependent_variable_Cat_TNE = self.merged_df[['Cat_TNE']]
        self.dependent_variable_No_more_episode = self.merged_df[['No_more_episode']]
        self.dependent_variable_TNE_desc = self.merged_df[['TNE_desc']]
        
        self.independent_variable = self.merged_df[['MiddleChildhood', 'Preschooler', 'Teenager', 'F', 'M', 'num_diagnoses',
       'num_medications', 'Therapy_ratio',
       'Examination_ratio', 'Advisory_ratio', 'TreatmentPlanning_ratio',
       'Outpatient_ratio', 'Inpatient_ratio', 'Inpatient_day_ratio',
       'Inpatient_daynight_ratio', 'closingcode_0', 'closingcode_1',
       'closingcode_2', 'closingcode_3', 'closingcode_4', 'closingcode_5',
       'closingcode_6', 'closingcode_9', 'aftercode_1', 'aftercode_2',
       'aftercode_3', 'aftercode_4', 'aftercode_5','Length_of_Episode', 'Count_visit','3131', '29622', '3151', '313', '3001', '304', '312', '3132', '3133', '3009', '3052', '2781', '1004', '3003', '2972', '3152', '3153', 
       '1019', '563', '2501', '315', '495', '7981', '637', '75021', '798', '476', '3034', '939', '345', '969', '339', '3004', '340', '7142', '5994', '2961', '3008', '300', '3891', '2926', '3069', '20411', '2953', '781', '785', '2903', '343', '915', '561', '3012', '316', '292', '3031', '74711', '2951', '317', '949', '3689', 
       '3453', '301', '3678', '19111', '1002', '3781', '789', '3401', '327', '3679', '2911', '661', '3276', '3011', '3671', '296', '3033', '389', '74712', '7522', '2713', '7581', '3274', '34512', '3123', '7373', '1702', '34511', '871', '74713', '3711', '819', '75211', '3591', '2444', '5551', '2649', '6562', '788', '2914', '1023',
        'N06BA04', 'N06AB06', 'N06BA09', 'A06BA04', 'N06AB03', 'N05CH01', 'R06AD01', 'N05AX08', 'N05AH04', 'N06BA12', 'N05AX12', 'N03AX09', 'NO6BAO4', 'N05AH03', 'NO6BA04', 'N06AB10', 'N05AA02', 'N06AX03', 'N05CF01', 'A11EA', 'A12AX', 'N06AX16', 'N03AG01', 'N05BA01', 'N05AF03', 'N02CX02', 'N05CF02', 'R06AE07', 'N05BB01', 'N05BA04', 'N06AA04', 
        'N02CC01', 'G03AA07', 'N03AE01', 'G03AC09', 'N06AX11', 'N02CC03', 'A06AD65', 'G03AA09', 'N06AB04', 'NO6BAO9', 'H01BA02', 'N06AX12', 'B03AA07', 'N05CD02', 'A10BA02', 'A02BC05', 'C02AC02', 'A03FA01', 'N06AB05', 'N06BA02', 'S01AA01', 'D07BC01', 'D07AC13', 'J01FA10', 'G03AA13', 'N01BB20', 'G03AC06', 'A06AD11', 'G03AA12', 'N05AD01', 'S01GX02',
        'D10AD03', 'M01AE02', 'A02BA02', 'J01CF01', 'D07AA02', 'N02AA59', 'A02BC02', 'R01AD09', 'G03FB05', 'N03AX14', 'D07AB08', 'G03AD02', 'A02BC01', 'D07AC01', 'N02BA01', 'D01AC20', 'R01AC02', 'N03AF01', 'A12BA02', 'M01AE01', 'S01GX09', 'R05DA01', 'D06AX01', 'H02AB02', 'A12BA01', 'D07XC01', 'N02BE01', 'A11JB', 'S01AA13', 'C09AA02', 'C07AA05', 
        'J05AB01', 'D06AX05', 'G03CA03', 'G03AD01', 'N07BA03', 'D10AD53','D10AD01']]
        
        return self.merged_df, self.dependent_variable_tillnextepisode, self.dependent_variable_Cat_TNE,self.dependent_variable_No_more_episode,self.dependent_variable_TNE_desc,self.independent_variable
        
    def train_classifier_without_medication_diagnosis(self):
        dependent_variables = [self.dependent_variable_Cat_TNE,self.dependent_variable_No_more_episode,self.dependent_variable_TNE_desc]
        dependent_variable_names = {'self.dependent_variable_Cat_TNE': 'Cat_TNE', 'self.dependent_variable_No_more_episode': 'No_more_episode', 'self.TNE_desc': 'TNE_desc'}
        class_weights = {'Cat_TNE': {0:1, 1:18, 2:19, 3:20, 4:21, 5:22, 6:23, 7:24, 8:25, 9:26, 10:27, 11:28, 12:29, 13:30}, 'No_more_episode': {0:10, 1:10}, 'TNE_desc': {0:1, 1:9, 2:10}}
              
        #for dependent_variable in dependent_variables:
        for variable_name, dependent_variable in zip(dependent_variable_names.values(), dependent_variables):
            logistic_prediction_model = LogisticRegression(class_weight=class_weights[variable_name])
            X_train, X_test, y_train, y_test = train_test_split(self.independent_variable, dependent_variable, train_size=0.7)
            logistic_prediction_model.fit(X_train,y_train)
            y_pred = logistic_prediction_model.predict(X_test)
            
            # Checks if the output category for classification is binary or multiclass
            category_count = int(y_train.nunique())
            print(f'\n****** Evaluation result {variable_name} as dependent variable ******')
            # For binary class classification 
            if category_count == 2:
                accuracy = accuracy_score(y_test, y_pred)
                precision = precision_score(y_test, y_pred, average='binary')
                recall = recall_score(y_test, y_pred, average='binary')
                f1 = f1_score(y_test, y_pred, average='binary')
                
                print(f"Accuracy: {accuracy}")
                print(f"Precision: {precision}")
                print(f"Recall: {recall}")
                print(f"F1score: {f1}")
                print("Confused matrix:")
                conf_matrix = confusion_matrix(y_test, y_pred)
                tn, fp, fn, tp = conf_matrix.ravel()
                print(conf_matrix)
                print(f"True Negative (TN): {tn}")
                print(f"False Positive (FP): {fp}")
                print(f"False Negative (FN): {fn}")
                print(f"True Positive (TP): {tp}")
                print(classification_report(y_test, y_pred))
            
            # For binary class classification 
            else:
                accuracy = accuracy_score(y_test, y_pred)
                precision = precision_score(y_test, y_pred, average='weighted')
                recall = recall_score(y_test, y_pred, average='weighted')
                f1 = f1_score(y_test, y_pred, average='weighted')
                
                print(f"Accuracy: {accuracy}")
                print(f"Precision: {precision}")
                print(f"Recall: {recall}")
                print(f"F1score: {f1}")
                conf_matrix = confusion_matrix(y_test, y_pred)
                print("Confused matrix:")
                print(conf_matrix)
                print(classification_report(y_test, y_pred))
                
    def prediction_without_medication_diagnosis(self):
        dependent_variables = [self.dependent_variable_tillnextepisode]
        dependent_variable_names = {'self.dependent_variable_tillnextepisode':'tillnextepisode'}
              
            
        #for dependent_variable in dependent_variables:
        for variable_name, dependent_variable in zip(dependent_variable_names.values(), dependent_variables):
            linear_prediction_model = LinearRegression()
            X_train, X_temp, y_train, y_temp = train_test_split(self.independent_variable, dependent_variable, train_size=0.7)
            X_test,X_eval,y_test,y_eval = train_test_split(X_temp,y_temp,test_size=0.33)
            linear_prediction_model.fit(X_train,y_train)
            y_pred = linear_prediction_model.predict(X_test)
            print(f'\n****** Prediction Model Evaluation result  {variable_name} as dependent variable ******')
            y_pred = linear_prediction_model.predict(X_test)
            r2 = r2_score(y_test, y_pred)
            mse = mean_squared_error(y_test, y_pred)
            print('R-squared:', r2)
            print('Mean squared error:', mse)
                

ClassifyReadmissionWithMedicationDiagnosis_Obj = ClassifyReadmissionWithMedicationDiagnosis('All_Full_ICD10_ATC.csv')
merged_df, dependent_variable_tillnextepisode, dependent_variable_Cat_TNE,dependent_variable_No_more_episode,dependent_variable_TNE_desc,independent_variable  = ClassifyReadmissionWithMedicationDiagnosis_Obj.load_data()

ClassifyReadmissionWithMedicationDiagnosis_Obj.train_classifier_without_medication_diagnosis()
ClassifyReadmissionWithMedicationDiagnosis_Obj.prediction_without_medication_diagnosis()
